# Colab Backend for AI Creative Platform
Run this to start a GPU-backed FastAPI server that mirrors your local API.


In [ ]:
!pip -q install -r colab_backend/requirements.txt


In [ ]:
import os
os.environ['HUGGINGFACE_TOKEN'] = os.getenv('HUGGINGFACE_TOKEN', '')
os.environ['CORS_ALLOWED_ORIGINS'] = os.getenv('CORS_ALLOWED_ORIGINS', 'http://localhost:5173')
print('CUDA available:', __import__('torch').cuda.is_available())


In [ ]:
# Import the necessary libraries
!pip -q install pyngrok

# NOTE: You MUST have your ngrok token stored in Colab Secrets 
# with the name 'NGROK_AUTH_TOKEN' for this to work.

from pyngrok import ngrok
from google.colab import userdata 
import subprocess
import sys

# --- 1. NGROK AUTHENTICATION ---
# Retrieve the token securely from Colab Secrets
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')

# Set the authentication token for pyngrok
if ngrok_token:
    ngrok.set_auth_token(ngrok_token)
else:
    print("ERROR: NGROK_AUTH_TOKEN not found in Colab Secrets. Please set it!")
    # If the token is missing, we stop here to avoid the error.
    # If you proceed without the token, the cell will crash with ERR_NGROK_4018
    # You may need to replace 'sys.exit()' with 'raise SystemExit()' or similar 
    # if you encounter issues within Colab's interactive environment.
    
# --- 2. CONNECT TO PORT 8000 ---
# Creates the public tunnel to the server we are about to launch.
public_url = ngrok.connect(8000, 'http')
print('Public URL:', public_url)

# --- 3. LAUNCH SERVER IN BACKGROUND ---
# Launches the Uvicorn server for the app located at 'colab_backend.app:app'
# It runs on the same port (8000) that ngrok is tunneling to.
# This subprocess runs in the background, allowing the cell to finish execution.
proc = subprocess.Popen(
    [
        sys.executable, 
        '-m', 
        'uvicorn', 
        'colab_backend.app:app', 
        '--host', 
        '0.0.0.0', 
        '--port', 
        '8000'
    ]
)

# Display the process object (useful for debugging, but the URL is the main output)
proc